In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [18]:
#week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Apr_list = [2, 9, 16, 23, 30]

In [19]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [20]:
len(link_day_minute_Apr_list)

14400

In [21]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [22]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [23]:
x[:,:2]

matrix([[ 1254.00148142,  1241.78943889],
        [ 1222.48812966,  1211.24697533],
        [ 1481.56003823,  1486.90013962],
        [ 1569.53102788,  1544.20614226],
        [ 1660.22116064,  1597.06527437],
        [ 1495.78271149,  1467.67259215],
        [ 1252.54023906,  1219.1787853 ],
        [ 1298.14408853,  1276.80390391],
        [ 1757.41097293,  1680.94036289],
        [ 1662.92852878,  1745.490104  ],
        [ 1514.50828613,  1585.54721941],
        [ 1251.18489395,  1233.25712411],
        [ 1173.15949751,  1200.34071961],
        [ 1368.04366583,  1358.26559671],
        [ 1224.49181757,  1159.10722546],
        [ 1059.03985511,  1041.02464249],
        [ 1220.97321887,  1197.89080601],
        [ 1191.17181189,  1224.46246572],
        [ 1577.88491398,  1562.60664768],
        [ 1627.90265065,  1646.15341691],
        [ 1470.81220719,  1497.64256652],
        [ 1419.91432544,  1420.21492615],
        [ 1286.41068927,  1242.64759027],
        [ 1308.96366346,  1268.736

In [24]:
np.size(A,0), np.size(A,1)

(24, 140)

In [25]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e+00, 4e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 27
 AA' NZ     : 3.510e+02
 Factor NZ  : 3.780e+02
 Factor Ops : 6.930e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -6.19945853e+07  0.00000000e+00  7.11e+03 1.20e+03  1.00e+06     0s
   1  -1.22459276e+07 -1.49119564e+07  3.08e+02 5.19e+01  1.51e+05     0s
   2  -1.06545465e+07 -1.17680081e+07  1.85e+01 3.11e+00  2.51e+04     0s
   3  -1.06649668e+07 -1.07751825e+07  1.85e-05 3.11e-06  1.9

In [26]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [27]:
lam_list

[1.052643395408399e-11,
 1310.9488674329448,
 1.155455579924171e-11,
 880.5964984862734,
 397.4048992383454,
 2.035738075676541,
 3.4868066443003174e-09,
 0.000513098932623822,
 0.07555405582935837,
 1279.0054611069954,
 0.06517847146807809,
 0.06272191248592751,
 0.05334077656430747,
 450.5554327065889,
 1899.0786494414726,
 975.9610558016844,
 0.04969577262294722,
 671.4615126882144,
 1011.739293018169,
 1.5243463046532497,
 0.8891902225533208,
 915.6677417436642,
 683.7046904202209,
 207.6402161491076,
 0.02592264212876927,
 820.7249224426134,
 0.07941253611173948,
 1214.0464354710468,
 0.017957083788267436,
 0.023174350159238077,
 0.01878697304925503,
 1304.8917157191806,
 226.89611839430498,
 443.669373785987,
 794.2128079565716,
 2.6558243895366508e-06,
 2.65579320050515e-06,
 2.6558132562009423e-06,
 0.04859910801635969,
 555.4159328639475,
 1352.8208426690614,
 0.6781215169492896,
 7.179384902505561,
 0.26238462249706895,
 1292.7698549462998,
 0.04863137291133546,
 215.58957135